# Analyzing player behavior to predict usage of a Minecraft research server

## Introduction

### Background

Minecraft is a sandbox, open-world video game developed by Mojang Studios where players explore, build and interact in a three-dimensional virtual world made of blocks. Players can build structures, gather resources and craft items to aid their survival, either individually or with other players in a multiplayer server (Minecraft.net, 2023). This makes it an ideal platform for studying player behaviors in a video game.

A research group led by Professor Frank Wood wants to assess how people play video games. To do this, they have set up a Minecraft server where players' interactions are recorded in detailed session logs. However, in doing so they face challenges such as identifying which players are most valuable to recruit and ensuring they have the resources necessary. 

### Question

The question we want to answer is: **Can player characteristics such as age, gender, and gaming experience predict the total amount of gameplay data (total playtime) contributed by players on a Minecraft server?**

### Data Description

We will be using the `players.csv` and `sessions.csv` datasets provided by the research group to determine which groups/types of players are most valuable, those who tend to engage the most, to optimize the recruitment process and ensure they have the resources necessary. 

The data is structured as follows:

1. `players.csv`

- **player_id** (String): identifier for each player
- **age** (Integer): age
- **gender** (Categorical): gender (Male, Female, Other)
- **experience** (Categorical): gaming experience level (Amateur, Regular, Pro, Veteran)
- **subscribe** (Boolean): whether the player subscribed to the game-related newsletter

This dataset contains **34** unique playesrs.

2. `sessions.csv`

- **player_id** (String): identifier for each player
- **start** (String): start time of the session
- **end** (String): end time of the session

This dataset contains **56** recorded sessions, where some of which have missing end times.

We calculate total playtime per player (in minutes) by summing all valid session durations. This will be used as the response variable. The explanatory variables are **age**, **gender**, **experience**, and **subscribe**.

## Methods & Result

### Load data

## Discussion

Lorem ipsum dolor sit amet, consectetur adipiscing elit. Praesent vel ligula scelerisque, feugiat erat vel, tincidunt justo. Curabitur ac sapien nec justo tincidunt interdum. Integer nec libero nec nisi tincidunt vehicula. Fusce id velit ut libero pulvinar facilisis. Sed euismod, nisl eget consectetur fermentum, nisi libero ultricies nunc, nec tincidunt sapien eros nec libero. Nullam id felis ut elit tincidunt pharetra. Suspendisse potenti. Proin ac orci vel arcu tincidunt pharetra. Integer vel nisi nec justo dictum tincidunt ut ac purus.

## References

Landin, Per. “What is Minecraft.” Minecraft.net, 5 June 2023, www.minecraft.net/en-us/article/what-minecraft.

